### Clase mensaje

In [48]:
#Clase donde vamos a ir metiendo todos los fragmentos de cada mensaje
class Mensaje: 
    idmensaje = 0
    frame = ""
    principal = ""
    nbytes = ""
    ncolumnas = 0
    nfilas = 0
    matriz = []
    end = ""
    
    macdest = [] #Direccion MAC Destino
    macorig = [] #Dirección MAC Origen
    tipo = [] #Tipo de mensaje que utiliza
    versionip = [] #Version de ip utilizada
    ipversion = ""
   
    cabip = [] #Parámetros de la cabecera IP
    tipoprotocoloip = []#Tipo de protocolo que utiliza el mensaje IP
    headerchecksumip = []#Checksum de la cabecera ip
    ipdest = [] #Dirección IP Destino
    iporig = [] #Dirección IP Origen
    ipprotocol = ""
    esip = False
    
    tcpsourceport = [] #Puerto origen para el datagrama TCP
    tcpdestport = [] #Puerto destino para el datagrama TCP
    tcpsequencenumber = [] #Número de secuencia del datagrama TCP
    tcpacknowledgenumber = [] #Número de acknowledge del datagrama TCP
    tcpheaderlenght = [] #Longitud de cabecera del datagrama TCP
    tcpflag = [] #Flag del datagrama TCP
    tcpwinsizevalue = [] #Tamaño de ventana del datagrama TCP
    tcpchecksum = [] #Checksum del datagrama TCP
    tcpurgentpointer = [] #Urgent pinter del datagrama TCP
    tcpmaxsegmentsize = [] #Tamaño máximo del segmento del datagrama TCP
    tcpsackoption = [] #Opcion sack del datagrama TCP
    tcptimestamps = [] #Timestamos del datagrama TCP
    tcpnop = [] #Operacion no del datagrama TCP
    tcpwindowscale = [] #Escala de ventana del datagrama TCP
    flagtcp = ""
    estcp = False
    
    icmptype = [] #Tipo de mensaje ICMP
    icmpcode = [] #Código del mesaje ICMP
    icmpchecksum = [] #Checksum del ICMP
    iptype = [] #Tipo de Mensaje IP
    esicmp=False

### Iniciador

In [49]:
#Esta función es necesaria para que los mensajes no se solapen cuando se asignan las partes de cada mensaje. 
#Si no se inicializa, las siguientes variables contendrán los elementos de todos los mensajes.
#En otras palabras, MAC destino será una lista con todas las MAC destino halladas. Todos los apartados de cada mensaje serán iguales
def iniciador(m):
    m.macdest = [] #Direccion MAC Destino
    m.macorig = [] #Dirección MAC Origen
    m.tipo = [] #Tipo de mensaje que utiliza
    m.versionip = [] #Version de ip utilizada
    m.ipversion = ""
   
    
    m.cabip = [] #Parámetros de la cabecera IP
    m.tipoprotocoloip = []#Tipo de protocolo que utiliza el mensaje IP
    m.headerchecksumip = []#Checksum de la cabecera ip
    m.ipdest = [] #Dirección IP Destino
    m.iporig = [] #Dirección IP Origen
    m.ipprotocol = ""
    m.esip = False
        
    m.tcpsourceport = [] #Puerto origen para el datagrama TCP
    m.tcpdestport = [] #Puerto destino para el datagrama TCP
    m.tcpsequencenumber = [] #Número de secuencia del datagrama TCP
    m.tcpacknowledgenumber = [] #Número de acknowledge del datagrama TCP
    m.tcpheaderlenght = [] #Longitud de cabecera del datagrama TCP
    m.tcpflag = [] #Flag del datagrama TCP
    m.tcpwinsizevalue = [] #Tamaño de ventana del datagrama TCP
    m.tcpchecksum = [] #Checksum del datagrama TCP
    m.tcpurgentpointer = [] #Urgent pinter del datagrama TCP
    m.tcpmaxsegmentsize = [] #Tamaño máximo del segmento del datagrama TCP
    m.tcpsackoption = [] #Opcion sack del datagrama TCP
    m.tcptimestamps = [] #Timestamos del datagrama TCP
    m.tcpnop = [] #Operacion no del datagrama TCP
    m.tcpwindowscale = [] #Escala de ventana del datagrama TCP
    m.flagtcp = ""
    m.estcp = False
    
    m.icmptype = [] #Tipo de mensaje ICMP
    m.icmpcode = [] #Código del mesaje ICMP
    m.icmpchecksum = [] #Checksum del ICMP
    m.iptype = [] #Tipo de Mensaje IP
    m.esicmp=False

#### Parseo de los elementos del archivo carrays

In [60]:
f = open("conexion2tcp", "r") #Abrimos el documento generado
mensajesarchivo = [] #Lista donde guardaremos todos los mensajes del archivo
acabado = False #Booleano para idicar cuando hemos acabado de leer el archivo
contador = 0 #Contador que nos indica el número de mensaje que estamos leyendo
while (acabado == False):
    msg = Mensaje() #asignamos a msg la estructura del mensaje
    msg.idmensaje = contador #asignamos el id que es el número de mensaje que estamos leyendo
    print(msg.idmensaje)
    msg.frame = f.readline() #Leemos la primera linea
    if (msg.frame == ""): #Si está vacío quiere decir que ya no hay más mensajes
        acabado = True
    else:    
        msg.principal = f.readline() #Leemos la descripción del paquete

        msg.nbytes = msg.frame[10]+msg.frame[11]+msg.frame[12] #Obtenemos el número de bytes de la primera línea
        msg.nbytes = int(msg.nbytes) #Convertimos el número a entero
        msg.ncolumnas = 8 #El número de columnas siempre es 8
        msg.nfilas = int(msg.nbytes/msg.ncolumnas) #Dividimos entre el numero de columnas para obtener el número de filas

        if (msg.nbytes%8 != 0): #Si el numero de bytes no es multiplo de 8, le añadimos una fila más
            msg.nfilas += 1

        #Iniciamos la matriz a none, para que tenga un valor
        msg.matriz = [None] * msg.nfilas #Iniciamos la matriz, con el número de filas 
        for i in range(msg.nfilas): #Completamos con el número de columnas
            msg.matriz[i] = [None]*msg.ncolumnas


        i=0
        #Bucle para obtener todos los datos del datagrama
        for linea in f:
            j=0 #Inicializamos j al comenzar la fila
            for caracter in linea.split(', '):
                if (j < 8):
                    msg.matriz[i][j] = caracter[0:4] #Para que coja solamente los 4 caracteres que queremos (se usa para el último caso sobre todo)
                    j+=1 #Incrementamos numero de columnas

            i+=1 #Incrementamos numero de filas
            if(i==msg.nfilas): #Si el numero de filas es igual al limite, paramos la lectura
                break
        #print (msg.matriz)####CHIVATO
        msg.end = f.readline() #Cogemos el }; final
        f.readline()#Salto de línea
        mensajesarchivo.append(msg) #Añadimos el mensaje a la lista de mensajes
        contador += 1
        
f.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


### Elementos del mensaje

In [61]:
def elementos(m):
    print("ID MENSAJE: ",m.idmensaje)
    print("Es TCP: ",m.estcp)
    print("Es ICMP: ",m.esicmp)
    print("Es IP: ",m.esip)
    print ("MAC Destino: ",m.macdest)
    print ("MAC Origen: ",m.macorig)
    print ("Tipo MAC: ",m.tipo)
    print ("VersionIP: ",m.versionip)
    print ("Cabecera IP: ",m.cabip)
    print ("Protocolo IP: ",m.tipoprotocoloip)
    print ("Checksum cabecera IP: ",m.headerchecksumip)
    print ("IP Destino: ",m.ipdest)
    print ("IP Origen: ",m.iporig)
    print ("Tipo IP: ",m.iptype)
    print ("Tipo ICMP: ",m.icmptype)
    print ("Código ICMP: ",m.icmpcode)
    print ("Checksum ICMP: ",m.icmpchecksum)
    print("Puerto origen TCP: ",m.tcpsourceport)
    print("Puerto destino TCP: ",m.tcpdestport)
    print("Número de secuencia TCP: ",m.tcpsequencenumber)
    print("Numero de ACK TCP: ",m.tcpacknowledgenumber)
    print("Longitud de cabecera TCP: ",m.tcpheaderlenght)
    print("Flag TCP: ",m.tcpflag)
    print("Tamaño de ventana TCP: ",m.tcpwinsizevalue)
    print("Checksum TCP: ",m.tcpchecksum)
    print("Urgent pointer TCP: ",m.tcpurgentpointer)
    print("Tamaño máximo de segmento TCP: ",m.tcpmaxsegmentsize)
    print("Opción Sack TCP: ",m.tcpsackoption)
    print("Timestamps TCP: ",m.tcptimestamps)
    print("NoOperation TCP: ",m.tcpnop)
    print("Escala de ventana TCP: ",m.tcpwindowscale)
    print("-------------------------------------------------")
    print(" ")

### Datagrama ICMP

In [62]:
def protocoloicmp(ipversion,contador, matriz, i,j):
    if ipversion == '4':
        if contador < 35:
            m.iptype.append(matriz[i][j])
        elif contador < 36:
            m.icmptype.append(matriz[i][j])
        elif contador < 37:
            m.icmpcode.append(matriz[i][j])
        elif contador < 39:
            m.icmpchecksum.append(matriz[i][j])
    elif ipversion== '6':
        if contador < 55:
            m.iptype.append(matriz[i][j])
        elif contador < 56:
            m.icmptype.append(matriz[i][j])
        elif contador < 57:
            m.icmpcode.append(matriz[i][j])
        elif contador < 59:
            m.icmpchecksum.append(matriz[i][j])

### Flags TCP

In [63]:
def flagstcp(contador, matriz, i, j):
    if contador < 50:
        m.tcpwinsizevalue.append(matriz[i][j]) 
    elif contador < 52:
        m.tcpchecksum.append(matriz[i][j])
    elif contador < 54:
        m.tcpurgentpointer.append(matriz[i][j])
    if m.flagtcp == '02' and contador >= 54:
        if contador < 58:
            m.tcpmaxsegmentsize.append(matriz[i][j])
        elif contador < 60:
            m.tcpsackoption.append(matriz[i][j])
        elif contador < 70:
            m.tcptimestamps.append(matriz[i][j])
        elif contador < 71:
            m.tcpnop.append(matriz[i][j])
        elif contador < 74:
            m.tcpwindowscale.append(matriz[i][j])

### Datagrama TCP

In [64]:
def protocolotcp(contador, matriz, i,j):
    if contador < 36:
        m.tcpsourceport.append(matriz[i][j])
    elif contador < 38:
        m.tcpdestport.append(matriz[i][j])
    elif contador < 42:
        m.tcpsequencenumber.append(matriz[i][j]) 
    elif contador < 46:
        m.tcpacknowledgenumber.append(matriz[i][j]) 
    elif contador < 47:
        m.tcpheaderlenght.append(matriz[i][j])    
    elif contador < 48:
        m.tcpflag.append(matriz[i][j])
        m.flagtcp = matriz[i][j][2]+matriz[i][j][3]
    else:
        flagstcp(contador, matriz, i, j)
        

### Datagrama IP

In [65]:
def protocoloip (ipversion,contador, matriz, i,j):
    if ipversion == '4': #Tratamos si usa ipv4
        if contador < 23:
            m.cabip.append(matriz[i][j])
        elif contador < 24:
            m.tipoprotocoloip.append(matriz[i][j])
            m.ipprotocol = matriz[i][j][3]
        elif contador < 26:
            m.headerchecksumip.append(matriz[i][j])
        elif contador < 30:
            m.iporig.append(matriz[i][j])
        elif contador < 34:
            m.ipdest.append(matriz[i][j])
        else:
            if m.ipprotocol == '6':
                m.estcp=True
                protocolotcp(contador, matriz, i,j)
            else:
                m.esicmp=True
                protocoloicmp(ipversion,contador, matriz,i,j) 
    elif ipversion == '6': #Tratamos si usa ipv6
        if contador < 22:
            m.cabip.append(matriz[i][j])
        elif contador < 38:
            m.iporig.append(matriz[i][j])
        elif contador < 54:
            m.ipdest.append(matriz[i][j])
        else:    
            m.protocoloicmp(contador, matriz,i,j)      

#### Separación de cabeceras y elementos del datagrama

In [67]:
n = 0 #Número de mensaje que estamos tratando

for m in mensajesarchivo: #Vamos leyendo cada mensaje
    contador = 0 #Contador para ir obteniendo los bytes del mensaje
    i = 0
    iniciador(m) #iniciamos los parámetros que se van a usar para que no se solapen  
    #Recorremos el mesaje extraido y lo vamos separando según los bytes que utiliza
    for i in range(m.nfilas):
        j = 0
        for j in range(m.ncolumnas):
            if contador < 6:
                m.macdest.append(m.matriz[i][j])
            elif contador < 12: 
                m.macorig.append(m.matriz[i][j])
            elif contador < 14:  
                m.tipo.append(m.matriz[i][j])
            elif contador < 15:
                m.versionip.append(m.matriz[i][j])
                m.ipversion = m.matriz[i][j][2]
            else:
                #m.esip = True
                protocoloip(m.ipversion, contador, m.matriz, i,j)   
            contador += 1 
    elementos(m)

ID MENSAJE:  0
Es TCP:  True
Es ICMP:  False
Es IP:  False
MAC Destino:  ['0x08', '0x00', '0x27', '0xac', '0xfd', '0x3d']
MAC Origen:  ['0x08', '0x00', '0x27', '0xdd', '0x97', '0x2e']
Tipo MAC:  ['0x08', '0x00']
VersionIP:  ['0x45']
Cabecera IP:  ['0x00', '0x00', '0x3c', '0x56', '0x9c', '0x40', '0x00', '0x40']
Protocolo IP:  ['0x06']
Checksum cabecera IP:  ['0x60', '0xc4']
IP Destino:  ['0xc0', '0xa8', '0x01', '0x0a']
IP Origen:  ['0xc0', '0xa8', '0x01', '0x01']
Tipo IP:  []
Tipo ICMP:  []
Código ICMP:  []
Checksum ICMP:  []
Puerto origen TCP:  ['0xde', '0x2a']
Puerto destino TCP:  ['0x00', '0x50']
Número de secuencia TCP:  ['0xf3', '0x4c', '0x3c', '0xe3']
Numero de ACK TCP:  ['0x00', '0x00', '0x00', '0x00']
Longitud de cabecera TCP:  ['0xa0']
Flag TCP:  ['0x02']
Tamaño de ventana TCP:  ['0x39', '0x08']
Checksum TCP:  ['0x12', '0x75']
Urgent pointer TCP:  ['0x00', '0x00']
Tamaño máximo de segmento TCP:  ['0x02', '0x04', '0x05', '0xb4']
Opción Sack TCP:  ['0x04', '0x02']
Timestamps TCP:

#### Pseudocódigo

##### Definir reglas

In [68]:
import os
tcprules=[]
udprules=[]
iprules=[]
icmprules=[]
def reglas():
    archivoreglas = os.listdir("/etc/snort/rules/") #Recorre todo el directorio
    for archivo in archivoreglas:
        ruta = "/etc/snort/rules/" + archivo #Nos metemos en un fichero del directorio
        f = open(ruta, "r") #Abrimos el documento generado
        for linea in f:
            if (linea[0] != "#" and linea != '\n'):
                if(linea[6:8] == "ip"):
                    iprules.append(linea)
                elif(linea[6:10] == "icmp"):
                    icmprules.append(linea)
                elif(linea[6:9]=="udp"):
                    udprules.append(linea)
                elif(linea[6:9]=="tcp"):
                    tcprules.append(linea)    
        f.close() 
    
    print(len(iprules))
    print(len(icmprules))
    print(len(tcprules))
    print(len(udprules))
reglas()

25
136
3500
288


In [69]:
#Comparar el mesaje con las reglas y que se activen cuando haya match

In [70]:
def analisis(m):
    if (m.esip):
        print("Es un mensaje IP")
    if(m.esicmp):
        icmp(m)
    if (m.estcp):
        print("Es un mensaje TCP")
        #Mirar direcciones origen destino, tamaño campo datos, ttl
        #alerta = "Mensaje ICMP con destino x y origen y"
    
        

In [98]:
def icmp(m)
for rule in icmprules:
    msg = rule.split(';')
    msg = msg[0].split('"')
    msg = msg[1]
    regla = rule.split()
    
    if regla[2] == "any" or regla[5]== "any":
        print(msg)

ICMP Destination Unreachable Communication Administratively Prohibited
ICMP Destination Unreachable Communication with Destination Host is Administratively Prohibited
ICMP Destination Unreachable Communication with Destination Network is Administratively Prohibited
PING Flooding


In [71]:
for m in mensajesarchivo:
    analisis(m)
    print()


Es un mensaje TCP

Es un mensaje TCP

Es un mensaje TCP

Es un mensaje TCP

Es un mensaje TCP

Es un mensaje TCP

Es un mensaje ICMP

Es un mensaje ICMP

Es un mensaje ICMP

Es un mensaje ICMP

Es un mensaje ICMP

Es un mensaje ICMP

Es un mensaje ICMP

Es un mensaje ICMP



## Por hacer
+ Añadir todos los tipos de mensajes existentes
+ Comparar reglas con mensajes y que envíen alertas (En forma de mensaje):
    - Mirar campos de la práctica de snort
    - Intentar hacer mach con esos campos (entre mensaje y alerta predefinida)


## Hecho
- Cambiar la manera de leer los archivos para que los lean enteros
- Crear una clase y una constructora para los mensajes
+ Añadir reglas estándar